# Cultivo de Modelos: Guia Prático

Este notebook demonstra como usar a API do **Preloto** para treinar, salvar e reutilizar modelos (Snapshots).
Isso é essencial para modelos pesados como Transformers e CatBoost.

In [ ]:
# Configuração Inicial
import os
import pandas as pd

# --- INSTALAÇÃO (Descomente uma das opções) ---

# Opção A: Repositório PÚBLICO (Mais simples)
# !pip install git+https://github.com/aretw0/predicoes-loterias-caixa.git

# Opção B: Repositório PRIVADO (Requer Token)
# 1. Adicione seu Token nas 'Sessions Secrets' do Colab (ícone de chave 🗝️ na esquerda) com nome 'GITHUB_TOKEN'
# 2. Habilite o acesso ao notebook nas configurações da Secret
# try:
#     from google.colab import userdata
#     token = userdata.get('GITHUB_TOKEN')
#     !pip install git+https://{token}@github.com/aretw0/predicoes-loterias-caixa.git
# except Exception as e:
#     print(f"Erro de Instalação: {e}. Verifique se adicionou a Secret 'GITHUB_TOKEN'.")

# Garantir diretório de snapshots
if not os.path.exists('snapshots'):
    os.makedirs('snapshots')
    print("Pasta 'snapshots/' criada.")

# Importando diretamente do pacote instalado
try:
    from src.loterias.megasena import MegaSena
    from src.loterias.models import TransformerModel, AutoEncoderModel
    print("✅ Módulos instalados.")
except ImportError:
    print("❌ Módulos não encontrados! Rode a instalação acima.")

## 1. Carregamento de Dados
Vamos usar a Mega Sena como exemplo.

In [ ]:
lottery = MegaSena()
df = lottery.preprocess_data()
print(f"Dados carregados: {len(df)} concursos.")
df.tail()

## 2. Cultivo (Treinamento)
Vamos treinar um **Transformer**. Isso geralmente demora, então é o candidato ideal para Snapshot.

In [ ]:
model = TransformerModel(range_min=1, range_max=60, draw_count=6)

# Treinamento (Reduzido para demonstração - use epochs=50+ na vida real)
print("Iniciando treinamento...")
history = model.train(df, epochs=5, batch_size=32)
print("Treinamento concluído.")

## 3. Snapshot (Salvamento)
Salvamos o modelo em disco para não perder o trabalho.

In [ ]:
filename = "snapshots/demo_transformer.keras"
model.save(filename)
print(f"Modelo salvo em {filename}")

## 4. Colheita (Carregamento e Uso)
Agora simulamos uma nova sessão. Criamos uma instância vazia e carregamos os pesos.

In [ ]:
# Instância Fresca
new_model = TransformerModel(1, 60, 6)

# Carregar
new_model.load(filename)
print("Modelo carregado com sucesso!")

# Predizer (passamos o df para ele pegar o contexto histórico recente)
prediction = new_model.predict(count=6, data=df)
print(f"Palpite do Modelo Carregado: {prediction}")

## 5. Validação com AutoEncoder
Podemos usar um segundo modelo para julgar o palpite.

In [ ]:
# Treinar o Fiscal
validator = AutoEncoderModel(1, 60, 6)
validator.train(df, epochs=5, verbose=0)

# Validar o palpite anterior
score = validator.validate(prediction)
print(f"Anomaly Score do palpite: {score:.6f}")

if score < 0.1:
    print("✅ Palpite APROVADO pelo Fiscal.")
else:
    print("❌ Palpite REJEITADO (Muito anômalo).")

In [ ]:
# 6. Baixar Snapshots (Google Colab)
# Compacta e baixa a pasta de snapshots automaticamente
import shutil
try:
    from google.colab import files
    shutil.make_archive('snapshots_backup', 'zip', 'snapshots')
    files.download('snapshots_backup.zip')
except ImportError:
    print("Ambiente não é o Colab ou google.colab não encontrado.")